In [24]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Nuestra eleccion fue la construccion de un modelo de recomendacion item-item, basado en la 'Similitud del coseno'

In [2]:
# Llamamos al dataframe que vamos a utilizar
archivo = pd.read_csv('../archivos_csv/ETL2_steam_games.csv')

In [3]:
# Seleccionar las columnas necesarias
df = archivo.loc[:, ["genres", "id", "app_name"]]

# Convertir 'item_id' a tipo entero y 'genres' a tipo str
df["id"] = df["id"].astype(int)
df['genres'] = df['genres'].astype(str)

# renombramos la columna 'id' a 'item_id' para una mayor intuicion
df = df.rename(columns= {'id': 'item_id'})

In [4]:
# agrupamos los valores de acuerdo a su item_id y a su nombre, a su vez unimos los valores agrupados en la columna 'genres' mediante el caracter ',' para luego ser utilizados en nuestro modelo
df = df.groupby(by= ['item_id','app_name'])['genres'].agg(lambda x: ', '.join(x)).reset_index()

In [5]:
df['genres'].value_counts()

Action                                                                     1878
Action, Indie                                                              1649
Simulation                                                                 1396
Casual, Simulation                                                         1359
Action, Adventure, Indie                                                   1082
                                                                           ... 
Action, Adventure, RPG, Indie                                                 1
Action, Adventure, Massively Multiplayer, Simulation, Early Access            1
Casual, RPG, Simulation, Early Access                                         1
Free to Play, Indie, Massively Multiplayer, RPG, Strategy, Early Access       1
Action, Adventure, Indie, Massively Multiplayer                               1
Name: genres, Length: 884, dtype: int64

In [6]:
# Crear un vectorizador de texto
cv = CountVectorizer()
vectores = cv.fit_transform(df['genres']).toarray()

In [7]:
# Calcular la similitud del coseno entre vectores
similitud = cosine_similarity(vectores)

In [8]:
# Generar una función para obtener recomendaciones por título
def recomendacion(juego):
    #Se busca el índice del juego en el DataFrame original (df). Este índice es utilizado para acceder a la fila correspondiente en la matriz de similitud.
    indice_juego = df[df["item_id"] == juego].index[0]
    
    #Cálculo de Similitudes: Se obtienen las distancias de similitud entre el juego de entrada y todos los demás juegos en el conjunto de datos. 
    distances = similitud[indice_juego]
    
    #Las distancias se ordenan de manera descendente, y se seleccionan los cinco juegos más similares (excluyendo el juego de entrada)
    lista_juegos = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    #Salida. Devuelve la lista de títulos recomendados.
    recommended_titles = [df.iloc[i[0]]['app_name'] for i in lista_juegos]
    
    return recommended_titles

In [9]:
# Aplicar la función a la columna 'item_id' y crear una nueva columna 'Recomendaciones'
df['Recomendaciones'] = df['item_id'].apply(recomendacion)

In [10]:
# borramos la columna 'app_name' ya que no la vamos a usar
df.drop(columns= ['app_name', 'genres'], inplace= True)

In [11]:
df

,item_id,genres,Recomendaciones
0,-1,"Action, Adventure","[Portal 2, Lost Planet™: Extreme Condition, To..."
1,10,Action,"[Team Fortress Classic, Day of Defeat, Deathma..."
2,20,Action,"[Team Fortress Classic, Day of Defeat, Deathma..."
3,30,Action,"[Team Fortress Classic, Day of Defeat, Deathma..."
4,40,Action,"[Team Fortress Classic, Day of Defeat, Deathma..."
...,...,...,...
28845,2028055,Action,"[Team Fortress Classic, Day of Defeat, Deathma..."
28846,2028056,Strategy,"[Space Empires IV Deluxe, Jagged Alliance 2 Go..."
28847,2028062,Action,"[Team Fortress Classic, Day of Defeat, Deathma..."
28848,2028103,"Action, Adventure","[Portal 2, Lost Planet™: Extreme Condition, To..."


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28850 entries, 0 to 28849
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   item_id          28850 non-null  int64 
 1   genres           28850 non-null  object
 2   Recomendaciones  28850 non-null  object
dtypes: int64(1), object(2)
memory usage: 676.3+ KB


In [21]:
df.drop(columns= 'genres', inplace= True)

In [22]:
# guardamos nuestro dataframe en un archivo .csv
df.to_csv('recomendaciones_item_item.csv', index= False)

In [25]:
# guardamos tambien en formato parquet para reducir el tamaño del archivo y que la ejecucion del modelo sea lo mas optima posible
pq.write_table(pa.Table.from_pandas(df), 'recomendaciones_item_item.parquet')